In [1]:
import os
import json
import requests
import pandas as pd
from pathlib import Path
from pprint import pprint
# Import API Key
from config import api_key


In [3]:
url = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.geojson?'

limit = 'limit=all'
country = 'country=US&'
type = 'fuel_type=ELEC&'
api = 'api_key=' + api_key + '&'
state = 'state=MD,VA,DC&'

response = requests.get(url + api + type + country + state + limit).json()
data = json.dumps(response,
                         indent = 2,
                         separators =(". ", " = "),
                         sort_keys = True)
print(data)
features = response['features']

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [5]:
latitude = []
longitude = []
date_confirmed = []
city = []
id = []
station_name = []
zip = []
street_address = []
dc_fast = []
level1 = []
level2 = []
public = []
owner = []
state = []
open_date = []
pricing = []
ev_connector_types = []

In [7]:
x = 0
for item in features:
        latitude.append(features[x]['geometry']['coordinates'][1])
        longitude.append(features[x]['geometry']['coordinates'][0])
        date_confirmed.append(features[x]['properties']['date_last_confirmed'])
        city.append(features[x]['properties']['city'])
        id.append(features[x]['properties']['id'])
        station_name.append(features[x]['properties']['station_name'])
        zip.append(features[x]['properties']['zip'])
        street_address.append(features[x]['properties']['street_address'])
        dc_fast.append(features[x]['properties']['ev_dc_fast_num'])
        level1.append(features[x]['properties']["ev_level1_evse_num"])
        level2.append(features[x]['properties']["ev_level2_evse_num"])
        public.append(features[x]['properties']["access_code"])
        owner.append(features[x]['properties']['ev_network'])
        state.append(features[x]['properties']['state'])
        open_date.append(features[x]['properties']['open_date'])
        pricing.append(features[x]['properties']['ev_pricing'])
        ev_connector_types.append(features[x]['properties']['ev_connector_types'][0])
        x = x + 1

In [9]:
df_data = {"ID":id,"Station_Name":station_name,"Access":public,"Owner":owner,"Last_Used":date_confirmed,"Open_Date":open_date,"DC_Fast":dc_fast,"Level1_Charging":level1, \
"Level2_Charging":level2, "Address":street_address,"City":city,"State":state,"Zipcode":zip,"Latitude":latitude,"Longitude":longitude,"Pricing":pricing,\
          "Connection_Type":ev_connector_types}

In [11]:
df = pd.DataFrame(df_data)
df.head()

,ID,Station_Name,Access,Owner,Last_Used,Open_Date,DC_Fast,Level1_Charging,Level2_Charging,Address,City,State,Zipcode,Latitude,Longitude,Pricing,Connection_Type
0,36363,ARUNDEL MILLS ELEMENT HOTELS,public,ChargePoint Network,2024-10-11,2010-07-15,NaN,1.0,NaN,7522 Teague Rd,Hanover,MD,21076,39.155907,-76.717216,None,NEMA515
1,39514,Hotel Floyd,public,Non-Networked,2022-11-10,2011-06-01,NaN,NaN,1.0,120 Wilson St,Floyd,VA,24091,36.910575,-80.317360,Free for guests; $10 for non-guests,J1772
2,39763,Passport Nissan - Alexandria,public,Non-Networked,2024-02-12,2011-03-15,1.0,NaN,3.0,150 S Pickett St,Alexandria,VA,22304,38.809670,-77.122190,Free,CHADEMO
3,39764,Priority Nissan - Chantilly,public,Non-Networked,2024-02-12,2011-03-15,1.0,NaN,2.0,14840 Stonecroft Center Ct,Chantilly,VA,20151,38.899800,-77.460330,Free,CHADEMO
4,39765,Colonial Nissan,public,Non-Networked,2024-02-12,2011-03-15,1.0,NaN,2.0,200 Myers Dr,Charlottesville,VA,22901,38.084926,-78.473415,Free,CHADEMO


In [13]:
df.dtypes

ID                   int64
Station_Name        object
Access              object
Owner               object
Last_Used           object
Open_Date           object
DC_Fast            float64
Level1_Charging    float64
Level2_Charging    float64
Address             object
City                object
State               object
Zipcode             object
Latitude           float64
Longitude          float64
Pricing             object
Connection_Type     object
dtype: object

In [15]:
df['Open_Date'] = pd.to_datetime(df['Open_Date'])
df['Last_Used'] = pd.to_datetime(df['Last_Used'])

In [17]:
df.dtypes

ID                          int64
Station_Name               object
Access                     object
Owner                      object
Last_Used          datetime64[ns]
Open_Date          datetime64[ns]
DC_Fast                   float64
Level1_Charging           float64
Level2_Charging           float64
Address                    object
City                       object
State                      object
Zipcode                    object
Latitude                  float64
Longitude                 float64
Pricing                    object
Connection_Type            object
dtype: object

In [19]:
df.to_csv('NREL_Data.csv', index=False)
